In [8]:
import numpy as np
import pandas as pd
import asyncio
import aiohttp
import json
import csv

In [160]:
import requests
import json
import time
from datetime import datetime
from datetime import date
import numpy as np
import pandas as pd

def get_markets(api = "cryptowatch"):
    """get markets available in API"""
    if api == "cryptowatch":
        try:
            r = requests.get("https://api.cryptowat.ch/markets").json()['result']
            exchanges = {item['exchange'] for item in r}
        except:
            print("Error: disfunctional API from Cryptowatch")
    return exchanges

def get_pairs(api = "cryptowatch"):
    """get pairs available in API"""
    if api == "cryptowatch":
        try:
            r = requests.get("https://api.cryptowat.ch/markets").json()['result']
            pairs = {item['pair'] for item in r}
        except:
            print("Error: disfunctional API from Cryptowatch")
    return pairs



In [168]:
def get_ohlc(market, pair, period = 24, api = "cryptowatch", before = "", after = "", local_timezone = True):
    """get OHLC prices available in API"""
    period *= 3600
    period = str(period)
    params = []
    url = "https://api.cryptowat.ch/markets/" + market + "/" + pair + "/ohlc"
    print(url)
    if (not before or not after):
        params = [('before', before), ('after', after)]
    if api == "cryptowatch":
        try:
            r = requests.get(url).json()['result'][period]
            #ohlc = [item['pair'] for item in r]
            if (local_timezone):            
                a = pd.DataFrame(r, columns = ['date(local)','open','high','low','close','volume','neg'])
                a[a.columns[0]] = pd.to_datetime(a[a.columns[0]] - time.timezone, unit = 's')
                return a
            else:
                a = pd.DataFrame(r, columns = ['date(utc)','open','high','low','close','volume','neg'])
                a[a.columns[0]] = pd.to_datetime(a[a.columns[0]], unit = 's')
                return a               
                
        except:
            print("Error: disfunctional API from Cryptowatch")
            
#columns = ['period','open','high','low','close','volume','neg']    

In [173]:
a = get_ohlc("bitfinex", "btcusd", period = 1, local_timezone = False)
#a.head()
b = get_ohlc("bitfinex", "btcusd", period = 1)
#b.head()
print(a[a.columns[0]][0:10])

print(b[b.columns[0]][0:10])


https://api.cryptowat.ch/markets/bitfinex/btcusd/ohlc
https://api.cryptowat.ch/markets/bitfinex/btcusd/ohlc
0   2018-06-26 02:00:00
1   2018-06-26 03:00:00
2   2018-06-26 04:00:00
3   2018-06-26 05:00:00
4   2018-06-26 06:00:00
5   2018-06-26 07:00:00
6   2018-06-26 08:00:00
7   2018-06-26 09:00:00
8   2018-06-26 10:00:00
9   2018-06-26 11:00:00
Name: date(utc), dtype: datetime64[ns]
0   2018-06-25 23:00:00
1   2018-06-26 00:00:00
2   2018-06-26 01:00:00
3   2018-06-26 02:00:00
4   2018-06-26 03:00:00
5   2018-06-26 04:00:00
6   2018-06-26 05:00:00
7   2018-06-26 06:00:00
8   2018-06-26 07:00:00
9   2018-06-26 08:00:00
Name: date(local), dtype: datetime64[ns]


In [174]:
a.tail()

,date(utc),open,high,low,close,volume,neg
495,2018-07-16 17:00:00,6635.0,6652.2,6620.0,6620.1,669.53260,4443321.0
496,2018-07-16 18:00:00,6620.5,6711.0,6620.3,6671.3,2524.33400,16872446.0
497,2018-07-16 19:00:00,6671.2,6703.7,6665.8,6671.1,821.73773,5490766.0
498,2018-07-16 20:00:00,6671.1,6689.6,6636.7,6681.1,857.29760,5707303.5
499,2018-07-16 21:00:00,6680.8,6680.9,6660.0,6665.8,205.49765,1370953.9


In [ ]:
np.random.random(5)

In [ ]:
e = get_markets()
print(e)

In [138]:
from datetime import date
print(date.fromtimestamp(1529978400))

2018-06-25


In [125]:
date.fromtimestamp(a['date'][0])

TypeError: an integer is required (got type str)